In [2]:
import pandas as pd
import numpy as np
from functools import reduce
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.feature_extraction import stop_words

In [3]:
#path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
cluster = pd.read_csv('/Users/kimia/Downloads/Clustered_Users.csv')

In [5]:
captions = pd.read_csv('/Users/kimia/Downloads/Caption_DF_22K.csv', encoding = "ISO-8859-1")

In [10]:
print(cluster.shape, captions.shape)

(163, 10) (21066, 4)


In [27]:
df = pd.merge(captions,cluster, on= 'User')

,Unnamed: 0_x,Caption,File,User,Unnamed: 0_y,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Prob_0,Prob_1,Prob_2,Prediction
0,0,Smart & easy @smarteez_official / New italian ...,1484832599818115504.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2
1,1,Favorite dress of the week Ã°ÂÂÂÃ°ÂÂÂ @e...,1523162357442196532.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2
2,2,@hellofashionblog in a total @fabianafilippi l...,1637923351476698851.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2
3,3,Our final SALE is now on Ã°ÂÂÂ 50% off ever...,1556601893983487653.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2
4,4,Antonello Ã°ÂÂÂ¤Ã°ÂÂÂ£ #ShopOnline #NewArr...,1531134122214665179.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2


In [210]:
"""
print(df.shape)
print(df.isnull().sum(axis=0))
#df = df.dropna()
del df['video_url']
print(df.shape)

metrics(df)"""

(248, 35)
(248, 35)
(248, 34)


In [265]:
"""line=hashtags[0]
' '.join(line)"""

'#getlost #instatravel #me #exploring #travelmemories #lifeofexploring #travelingram #girls #travelstories #passionpassport #travelgo #wanderlust #mytravelgram #lifeintravel #travelinspired #travelblogger #beautifuldestination #travelbloggerlife #travelstyle #shoot #travellingthroughtheworld #igtravel #Ig_Italy, #igersitalia #igercroazia'

# hashtags said by the community

In [19]:
## get hashtags
hashtags = []
for i,v in enumerate(df.Caption):
        v = str(v)
        #tokenizedTweet = list(re.findall(r'(?i)\#\w+', str))
        
        hashtag = []
        for t in v.split():
            if '#' in t:
                hashtag = hashtag + ['#' + x for x in t.split('#')[1:]]
        #hashtag  =[ t for t in v.split() if '#' in t ]
        #print(hashtag)
        hashtags.append(' '.join(hashtag))
        
df['hashtags'] = pd.Series(hashtags)

In [20]:
df.head()

,Unnamed: 0_x,Caption,File,User,Unnamed: 0_y,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Prob_0,Prob_1,Prob_2,Prediction,hashtags
0,0,Smart & easy @smarteez_official / New italian ...,1484832599818115504.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2,#SS2017
1,1,Favorite dress of the week Ã°ÂÂÂÃ°ÂÂÂ @e...,1523162357442196532.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2,#NewArrivals #ResortWear
2,2,@hellofashionblog in a total @fabianafilippi l...,1637923351476698851.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2,#FabianaFilippi #WinterChic #NewCollection #Mi...
3,3,Our final SALE is now on Ã°ÂÂÂ 50% off ever...,1556601893983487653.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2,#Clearance
4,4,Antonello Ã°ÂÂÂ¤Ã°ÂÂÂ£ #ShopOnline #NewArr...,1531134122214665179.jpg,_____bettina_____,0,7.949732e-47,0.006667,0.993333,2.293494e-120,0.000799,0.000021,0.99918,2,#ShopOnline #NewArrivals #ClutchBag


In [22]:

## THIS WORKS
Predictionx=[]
number = []
words = []
weightedNum = []
userNames = []
for i, v in enumerate(df.hashtags):
     try: 
        #print(i)
        #print(v)
        line = df.hashtags[i:i+1]
        
        df['polarity'] = 0

        vectorizer = CountVectorizer(stop_words=None)#min_df=4
        df['vector'] = vectorizer

        corpus = line.values
        x = vectorizer.fit_transform(corpus)
        x = x.toarray()[0]
        terms = vectorizer.get_feature_names()
        
        #userWeight = df.final_score[i]
        userName = df.User[i]
        Prediction = df.Prediction[i]
        assert len(terms) == len(x)
        for j in range(len(terms)):
            words.append(terms[j])
            number.append(x[j])
            userNames.append(userName)
            #weightedNum.append(x[j]*userWeight)
            Predictionx.append(Prediction)
    
     except ValueError:
        print('No hashtags in this post' ,i)
        words.append('0')
        number.append(0)
        userNames.append(userName)
        Predictionx.append(0)
        #weightedNum.append(0)
        
#hashCounts = pd.DataFrame({'word': words, 'number': number, 'weightedNum': weightedNum, 'userName': userNames})
hashCounts = pd.DataFrame({'word': words, 'number': number, 'userName': userNames, 'Prediction': Predictionx})
hashCounts



No hashtags in this post 64
No hashtags in this post 74
No hashtags in this post 117
No hashtags in this post 151
No hashtags in this post 154
No hashtags in this post 158
No hashtags in this post 160
No hashtags in this post 161
No hashtags in this post 167
No hashtags in this post 172
No hashtags in this post 174
No hashtags in this post 175
No hashtags in this post 176
No hashtags in this post 179
No hashtags in this post 181
No hashtags in this post 184
No hashtags in this post 185
No hashtags in this post 187
No hashtags in this post 188
No hashtags in this post 190
No hashtags in this post 191
No hashtags in this post 194
No hashtags in this post 196
No hashtags in this post 199
No hashtags in this post 209
No hashtags in this post 216
No hashtags in this post 217
No hashtags in this post 218
No hashtags in this post 221
No hashtags in this post 223
No hashtags in this post 225
No hashtags in this post 228
No hashtags in this post 230
No hashtags in this post 235
No hashtags in t

No hashtags in this post 782
No hashtags in this post 837
No hashtags in this post 860
No hashtags in this post 880
No hashtags in this post 900
No hashtags in this post 901
No hashtags in this post 903
No hashtags in this post 904
No hashtags in this post 906
No hashtags in this post 908
No hashtags in this post 912
No hashtags in this post 915
No hashtags in this post 916
No hashtags in this post 917
No hashtags in this post 921
No hashtags in this post 925
No hashtags in this post 927
No hashtags in this post 930
No hashtags in this post 933
No hashtags in this post 934
No hashtags in this post 936
No hashtags in this post 937
No hashtags in this post 942
No hashtags in this post 943
No hashtags in this post 944
No hashtags in this post 947
No hashtags in this post 949
No hashtags in this post 950
No hashtags in this post 954
No hashtags in this post 955
No hashtags in this post 956
No hashtags in this post 957
No hashtags in this post 959
No hashtags in this post 960
No hashtags in

No hashtags in this post 2229
No hashtags in this post 2230
No hashtags in this post 2231
No hashtags in this post 2234
No hashtags in this post 2235
No hashtags in this post 2239
No hashtags in this post 2241
No hashtags in this post 2242
No hashtags in this post 2371
No hashtags in this post 2388
No hashtags in this post 2394
No hashtags in this post 2396
No hashtags in this post 2397
No hashtags in this post 2398
No hashtags in this post 2399
No hashtags in this post 2400
No hashtags in this post 2404
No hashtags in this post 2405
No hashtags in this post 2407
No hashtags in this post 2408
No hashtags in this post 2411
No hashtags in this post 2416
No hashtags in this post 2419
No hashtags in this post 2421
No hashtags in this post 2423
No hashtags in this post 2425
No hashtags in this post 2426
No hashtags in this post 2428
No hashtags in this post 2431
No hashtags in this post 2432
No hashtags in this post 2435
No hashtags in this post 2438
No hashtags in this post 2440
No hashtag

No hashtags in this post 3558
No hashtags in this post 3562
No hashtags in this post 3563
No hashtags in this post 3565
No hashtags in this post 3566
No hashtags in this post 3567
No hashtags in this post 3568
No hashtags in this post 3570
No hashtags in this post 3571
No hashtags in this post 3572
No hashtags in this post 3573
No hashtags in this post 3574
No hashtags in this post 3576
No hashtags in this post 3577
No hashtags in this post 3578
No hashtags in this post 3579
No hashtags in this post 3580
No hashtags in this post 3581
No hashtags in this post 3582
No hashtags in this post 3584
No hashtags in this post 3586
No hashtags in this post 3587
No hashtags in this post 3588
No hashtags in this post 3589
No hashtags in this post 3590
No hashtags in this post 3591
No hashtags in this post 3592
No hashtags in this post 3593
No hashtags in this post 3594
No hashtags in this post 3595
No hashtags in this post 3596
No hashtags in this post 3598
No hashtags in this post 3599
No hashtag

No hashtags in this post 4807
No hashtags in this post 4809
No hashtags in this post 4811
No hashtags in this post 4812
No hashtags in this post 4814
No hashtags in this post 4816
No hashtags in this post 4819
No hashtags in this post 4821
No hashtags in this post 4822
No hashtags in this post 4823
No hashtags in this post 4828
No hashtags in this post 4829
No hashtags in this post 4830
No hashtags in this post 4831
No hashtags in this post 4833
No hashtags in this post 4834
No hashtags in this post 4837
No hashtags in this post 4839
No hashtags in this post 4840
No hashtags in this post 4843
No hashtags in this post 4847
No hashtags in this post 4848
No hashtags in this post 4851
No hashtags in this post 4852
No hashtags in this post 4854
No hashtags in this post 4855
No hashtags in this post 4858
No hashtags in this post 4859
No hashtags in this post 4862
No hashtags in this post 4863
No hashtags in this post 4866
No hashtags in this post 4867
No hashtags in this post 4868
No hashtag

...

No hashtags in this post 15485
No hashtags in this post 15486
No hashtags in this post 15487
No hashtags in this post 15490
No hashtags in this post 15491
No hashtags in this post 15493
No hashtags in this post 15494
No hashtags in this post 15495
No hashtags in this post 15496
No hashtags in this post 15497
No hashtags in this post 15498
No hashtags in this post 15500
No hashtags in this post 15501
No hashtags in this post 15502
No hashtags in this post 15505
No hashtags in this post 15510
No hashtags in this post 15511
No hashtags in this post 15513
No hashtags in this post 15519
No hashtags in this post 15520
No hashtags in this post 15524
No hashtags in this post 15525
No hashtags in this post 15527
No hashtags in this post 15529
No hashtags in this post 15530
No hashtags in this post 15531
No hashtags in this post 15533
No hashtags in this post 15534
No hashtags in this post 15537
No hashtags in this post 15538
No hashtags in this post 15539
No hashtags in this post 15540
No hasht

No hashtags in this post 16768
No hashtags in this post 16769
No hashtags in this post 16770
No hashtags in this post 16771
No hashtags in this post 16772
No hashtags in this post 16773
No hashtags in this post 16774
No hashtags in this post 16775
No hashtags in this post 16776
No hashtags in this post 16777
No hashtags in this post 16778
No hashtags in this post 16779
No hashtags in this post 16780
No hashtags in this post 16781
No hashtags in this post 16782
No hashtags in this post 16783
No hashtags in this post 16784
No hashtags in this post 16785
No hashtags in this post 16786
No hashtags in this post 16787
No hashtags in this post 16788
No hashtags in this post 16789
No hashtags in this post 16790
No hashtags in this post 16791
No hashtags in this post 16792
No hashtags in this post 16793
No hashtags in this post 16795
No hashtags in this post 16796
No hashtags in this post 16797
No hashtags in this post 16907
No hashtags in this post 17080
No hashtags in this post 17228
No hasht

No hashtags in this post 19158
No hashtags in this post 19578
No hashtags in this post 19632
No hashtags in this post 19718
No hashtags in this post 19755
No hashtags in this post 19759
No hashtags in this post 19761
No hashtags in this post 19779
No hashtags in this post 19799
No hashtags in this post 19821
No hashtags in this post 19825
No hashtags in this post 19837
No hashtags in this post 19838
No hashtags in this post 19839
No hashtags in this post 19840
No hashtags in this post 19841
No hashtags in this post 19842
No hashtags in this post 19843
No hashtags in this post 19844
No hashtags in this post 19845
No hashtags in this post 19847
No hashtags in this post 19848
No hashtags in this post 19849
No hashtags in this post 19851
No hashtags in this post 19853
No hashtags in this post 19854
No hashtags in this post 19856
No hashtags in this post 19857
No hashtags in this post 19858
No hashtags in this post 19860
No hashtags in this post 19862
No hashtags in this post 19864
No hasht

No hashtags in this post 20720
No hashtags in this post 20900
No hashtags in this post 20901
No hashtags in this post 20903
No hashtags in this post 20906
No hashtags in this post 20908
No hashtags in this post 20914
No hashtags in this post 20915
No hashtags in this post 20918
No hashtags in this post 20927
No hashtags in this post 20932
No hashtags in this post 20943
No hashtags in this post 20948
No hashtags in this post 20970
No hashtags in this post 20981
No hashtags in this post 20990
No hashtags in this post 21002
No hashtags in this post 21006
No hashtags in this post 21018
No hashtags in this post 21027
No hashtags in this post 21047
No hashtags in this post 21052
No hashtags in this post 21054


,Prediction,number,userName,word
0,2,1,_____bettina_____,ss2017
1,2,1,_____bettina_____,newarrivals
2,2,1,_____bettina_____,resortwear
3,2,1,_____bettina_____,fabianafilippi
4,2,1,_____bettina_____,midseasonsale
5,2,1,_____bettina_____,newcollection
6,2,1,_____bettina_____,winterchic
7,2,1,_____bettina_____,clearance
8,2,1,_____bettina_____,clutchbag
9,2,1,_____bettina_____,newarrivals


In [23]:
hashCounts.Prediction.unique()

array([2, 0, 1])

In [24]:
#hashCounts
for i,v in enumerate(hashCounts.Prediction.unique()):
    df = hashCounts[hashCounts.Prediction == v]
    df = df.groupby(['word']).sum().sort_values(by='number', ascending=False)
    Predic = 'Predict_Hashtags_By_Community_' + str(v) + '.csv'
    df.to_csv(Predic)
    

# words said by the community

In [28]:
df.columns

Index(['Unnamed: 0_x', 'Caption', 'File', 'User', 'Unnamed: 0_y', 'Cluster_0',
       'Cluster_1', 'Cluster_2', 'Cluster_3', 'Prob_0', 'Prob_1', 'Prob_2',
       'Prediction'],
      dtype='object')

In [29]:
Predictionx= []
## THIS WORKS
number = []
words = []
weightedNum = []
userNames = []
for i, v in enumerate(df.Caption):
     try: 
        #print(i)
        #print(v)
        line = df.Caption[i:i+1]
        
        df['polarity'] = 0

        vectorizer = CountVectorizer(stop_words=list(stop_words.ENGLISH_STOP_WORDS))#min_df=4
        df['vector'] = vectorizer

        corpus = line.values
        x = vectorizer.fit_transform(corpus)
        x = x.toarray()[0]
        terms = vectorizer.get_feature_names()
        
        #userWeight = df.final_score[i]
        userName = df.User[i]
        Prediction = df.Prediction[i]
        assert len(terms) == len(x)
        for j in range(len(terms)):
            words.append(terms[j])
            number.append(x[j])
            userNames.append(userName)
            #weightedNum.append(x[j]*userWeight)
            Predictionx.append(Prediction)
    
     except ValueError:
        print('nan present in the DF at point' ,i)
        words.append('0')
        number.append(0)
        userNames.append(userName)
        #weightedNum.append(0)
        Predictionx.append(0)
        
#wordCounts = pd.DataFrame({'word': words, 'number': number, 'weightedNum': weightedNum, 'userName': userNames})
wordCounts = pd.DataFrame({'word': words, 'number': number,  'userName': userNames, 'Prediction': Predictionx})

#wordCounts



nan present in the DF at point 161
nan present in the DF at point 340
nan present in the DF at point 343
nan present in the DF at point 466
nan present in the DF at point 494
nan present in the DF at point 521
nan present in the DF at point 527
nan present in the DF at point 564
nan present in the DF at point 590
nan present in the DF at point 596
nan present in the DF at point 996
nan present in the DF at point 1007
nan present in the DF at point 1046
nan present in the DF at point 1091
nan present in the DF at point 1093
nan present in the DF at point 1095
nan present in the DF at point 1667
nan present in the DF at point 1678
nan present in the DF at point 1743
nan present in the DF at point 1774
nan present in the DF at point 1790
nan present in the DF at point 2084
nan present in the DF at point 2115
nan present in the DF at point 2199
nan present in the DF at point 2225
nan present in the DF at point 2767
nan present in the DF at point 2784
nan present in the DF at point 3602
nan

In [30]:
#wordCounts.groupby(['word']).sum().sort_values(by='weightedNum', ascending=False)

#wordCounts
for i,v in enumerate(wordCounts.Prediction.unique()):
    df = wordCounts[wordCounts.Prediction == v]
    df = df.groupby(['word']).sum().sort_values(by='number', ascending=False)
    Predic = 'Predict_words_By_Community_' + str(v) + '.csv'
    df.to_csv(Predic)